# 4 задача
## Найти численное решение смешанной задачи для уравнения переноса в квадрате 0 < x,t < 1 и сравнить их значения в 11 точках при t  = 1

### Постановка задачи (вариант №1 задание №11)

#### Дифференциальная задача:

\begin{gather*}
   {\partial u \over \partial t}  + e^x {\partial u \over \partial x} = x\\
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

#### Краевые условия задачи:

\begin{equation}
\left.
  \begin{array}{ccc}
      u(x,0) = e^{2x} +(x-1)e^x \\
      u(0,t) = (t-1)^2-1
  \end{array}
\right\}
\end{equation}

#### Разностная схема:

\begin{equation}
\left\{
  \begin{array}{ccc}
  u_l^{n+1} = u_l^n-{\tau_n \over h}e^{-x_l}(u_l^n-u_{l-1}^n)+\tau_nx_l, \;\;\;\;\; l \in [1;L], \;\; n \in [0;N-1]\\
      u_l^0 = e^{2x_l}+(x_l-1)e^{x_l}, \;\;\;\;\; l \in [0;L]\\ 
      u_o^n = (t^n-1)^2-1, \;\;\;\;\; n \in [1;N]
  \end{array}
\right.
\end{equation}

In [93]:
u0, u1 = 0, 1            # концы отрезка
L = 10241                # число узлов
h = (u1 - u0) / (L - 1)  # шаг сетки
Nt = 10241                 # число шагов по времени
tau = 1 / (Nt - 1)      # шаг по времени

In [17]:
from math import *
from scipy.sparse import dia_matrix
#from numpy import linalg as LA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [94]:
#проверим спектральный признак устойчивости
print('tau = ', tau)
print('2h/a =', 2*h/1)

tau =  9.765625e-05
2h/a = 0.0001953125


In [95]:
#начальное условние
start = np.array([exp(2*i*h) + (i*h-1)*exp(i*h) for i in range(0,L)])
print(start.size)
t = 1

10241


In [46]:
#функция расчета следующего слоя по времени:
def NextLay(prevlay,time):
    bound = (time-1)**2-1
    other = [prevlay[i]-tau/h*exp(-i*h)*(prevlay[i]-prevlay[i-1])+tau*i*h for i in range(1,L)]
    return np.array([bound] + other)

In [75]:
NextLay(start,1*tau)[::128]

array([-1.56188965e-03,  2.25023786e-01,  5.12795906e-01,  8.75110524e-01,
        1.32811738e+00,  1.89134754e+00,  2.58842505e+00,  3.44793064e+00,
        4.50445010e+00,  5.79984760e+00,  7.38481271e+00])

In [96]:
currentlay = start
for j in range(1, Nt):
    currentlay = NextLay(currentlay,tau*j)
solution = currentlay
print(solution[::1024])

[-1.         -0.98358589 -0.92808392 -0.82246227 -0.65313871 -0.40341944
 -0.0528158   0.42378605  1.05710768  1.88479831  2.95286652]


In [60]:
sol32 = solution[::32]

In [70]:
sol64 = solution[::64]

In [78]:
sol128 = solution[::128]

In [85]:
sol256 = solution[::256]

In [91]:
sol512 = solution[::512]

In [97]:
sol1024 = solution[::1024]

In [98]:
# решения в 11 точках на [0;1]
pd.DataFrame([sol32,sol64,sol128,sol256,sol512,sol1024],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121,10241])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,-1.0,-0.983368,-0.927490,-0.821290,-0.651132,-0.400252,-0.048080,0.430481,1.065328,1.894630,2.964452
641,-1.0,-0.983480,-0.927797,-0.821896,-0.652169,-0.401889,-0.050527,0.427034,1.061086,1.889557,2.958475
1281,-1.0,-0.983537,-0.927950,-0.822198,-0.652686,-0.402706,-0.051748,0.425304,1.058965,1.887020,2.955484
2561,-1.0,-0.983565,-0.928027,-0.822349,-0.652945,-0.403114,-0.052358,0.424437,1.057904,1.885750,2.953989
5121,-1.0,-0.983579,-0.928065,-0.822425,-0.653074,-0.403318,-0.052663,0.424003,1.057373,1.885116,2.953241
10241,-1.0,-0.983586,-0.928084,-0.822462,-0.653139,-0.403419,-0.052816,0.423786,1.057108,1.884798,2.952867


In [104]:
err32 = abs((sol32-sol1024)/sol1024)
err64 = abs((sol64-sol1024)/sol1024)
err128 = abs((sol128-sol1024)/sol1024)
err256 = abs((sol256-sol1024)/sol1024)
err512 = abs((sol512-sol1024)/sol1024)

In [105]:
# относительная ошибка 
pd.DataFrame([err32,err64,err128,err256,err512],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,0.0,0.000222,0.000640,0.001425,0.003073,0.007850,0.089664,0.015798,0.007777,0.005216,0.003924
641,0.0,0.000107,0.000309,0.000689,0.001485,0.003794,0.043334,0.007663,0.003764,0.002525,0.001899
1281,0.0,0.000050,0.000144,0.000321,0.000693,0.001769,0.020210,0.003582,0.001757,0.001179,0.000887
2561,0.0,0.000021,0.000062,0.000138,0.000297,0.000758,0.008659,0.001537,0.000753,0.000505,0.000380
5121,0.0,0.000007,0.000021,0.000046,0.000099,0.000253,0.002886,0.000513,0.000251,0.000168,0.000127
